# Classificação de Sentimentos com Redes Neurais Convolucionais

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

import re
import math
from bs4 import BeautifulSoup
from google.drive import drive
import zipfile

In [3]:
test_data = pd.read_csv('../..//_IAExpert_private/7. Processamento de Linguagem Natural com Deep LEarning/trainingandtestdata/test.csv')
print(test_data.shape)
test_data.sample(5, random_state=0)

(497, 6)


,4,3,Mon May 11 03:17:40 UTC 2009,kindle2,tpryan,"@stellargirl I loooooooovvvvvveee my Kindle2. Not that the DX is cool, but the 2 is fantastic in its own right."
90,0,213,Mon May 25 17:25:34 UTC 2009,insects,malcozer,insects have infected my spinach plant :(
254,4,1020,Mon May 11 05:21:57 UTC 2009,lebron,BI_POLAROID,@Lou911 Lebron is MURDERING shit.
283,2,2101,Sun May 17 17:30:25 UTC 2009,san francisco,rayceja,Heading to San Francisco
443,2,2533,Sun Jun 07 01:13:34 UTC 2009,jquery,5x1llz,This is cold.. I was looking at google's chart...
336,4,2187,Mon May 25 17:29:06 UTC 2009,mcdonalds,connlocks,Just had McDonalds for dinner. :D It was goooo...
